In [ ]:
import yaml
from pathlib import Path
import sys
import numpy as np
import matplotlib.pyplot as plt

# Get project root and add to path
notebook_dir = Path.cwd()
project_root = Path('pytorch/crypto/classifier')
sys.path.insert(0, str(project_root))

# Import only what we need
from src.data_loader import download_crypto_data, create_price_sequences

# Load config
with open(project_root / 'config' / 'config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Download/load data
data_config = config['data']
cache_dir = project_root / config['paths']['raw_data']

data = download_crypto_data(
    symbol=data_config['symbol'],
    interval=data_config['interval'],
    start_date_str=data_config['start_date'],
    end_date_str=data_config['end_date'],
    cache_dir=str(cache_dir)
)

print(f"Loaded {len(data)} data points")

# Create sequences
seq_len = config['image']['seq_len']
closing_prices = data.values
sequences = create_price_sequences(closing_prices, seq_len)

print(f"Created {len(sequences)} sequences")

# Get first sequence
sequence = sequences[0]

# Normalize to [0, 1] range
seq_min, seq_max = sequence.min(), sequence.max()
if seq_max > seq_min:
    sequence_norm = (sequence - seq_min) / (seq_max - seq_min)
else:
    sequence_norm = np.full_like(sequence, 0.5)

# Create image using matplotlib
resolution = config['image']['resolution']
width, height = resolution['width'], resolution['height']
line_width = config['image']['line_width']

# Create figure with exact size
dpi = 100
fig, ax = plt.subplots(figsize=(width/dpi, height/dpi), dpi=dpi)

# Plot line (inverted y-axis for image coordinates)
x = np.linspace(0, width, len(sequence_norm))
y = (1 - sequence_norm) * height

ax.plot(x, y, color='black', linewidth=line_width)

# Set exact limits and remove all decorations
ax.set_xlim(0, width)
ax.set_ylim(0, height)
ax.set_aspect('equal')
ax.axis('off')

# Remove margins
plt.subplots_adjust(left=0, right=1, top=1, bottom=0)

# Set white background
fig.patch.set_facecolor('white')
ax.set_facecolor('white')

plt.show()

print(f"Displayed sequence with {len(sequence)} points")
print(f"Price range: [{seq_min:.2f}, {seq_max:.2f}]")